# Word Embeddings : le modèle Word2Vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [5]:
infile = f"../../data/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [6]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [7]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [8]:
len(bigram_phrases.vocab.keys())

15852820

Prenons une clé au hasard :

In [9]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

1q


Le dictionnaire indique le score de cette coocurrence :

In [10]:
bigram_phrases.vocab[key_]

488

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [11]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [12]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [13]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [17]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [18]:
print(corpus[:100])

[['mi', 'imnri', 'r', 'i', '<<', 'i', 'i', 'hmu', "'", 'i', '/', 'tx', "-'", 'l', ':', 'marche', 'tenu', 'hors', 'villa', ',', 'la', '9', '.'], ['--', 'u', 'a', 'ete', 'vaain', 'si', 'teicj', '>>', 'm', 'races_indigenes', 'de', 'fr', '.'], ['31', '<)', 'a', '5s', "'", 'k', '131', 'de', '.'], ['rasa', 'iichakdui', "'", 'te', ',', 'do', '(', 'r', '.', '3s0', 'h', '710', '.', 'taureaux', 'iallsenas', ',>', 'ia', 'u', '\\', '--', 'a', '--', ';', '0ii', '.'], ['hollandais', ',', 'dufr', '.'], ['0', '.'], ['--', 'a', '9', '.--', 'la', 'idto', '-', 'vachei', 'laitieres', ':', 'bn', 'vante', '1q', '.'], ['vendues', '3', '\\', 'au', 'prix', 'la', '410', 'a', '*', '<<', 'i', 'h', '\\;', 'genisses', ',', 'kl', '.'], ["'.", '9', '.'], ['i', 'l', '.', '2', 'i', '.', 'id', '.'], ['da', '370', 'i', '6lutr', '.'], ['marche', 'a', '<', 'u', 'porcs', '.'], ['--', 'categorie', 'de', 'lt', 'ilashtya', ':', "'", '237', 'on', 'vente', ';', 'vendus', '1', 'm', '.', 'do', "'", '2', 'i', '.--', 'a', ';:,', 'l'

## Entrainement d'un modèle Word2Vec sur ce corpus

In [19]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et 5 après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus (on peut le mettre à plus avec un très gros corpus)
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: total: 34min 23s
Wall time: 17min 41s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [21]:
outfile = f"../../data/newspapers.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [23]:
model = Word2Vec.load("../../data/newspapers.model")

### Imprimer le vecteur d'un terme

In [28]:
model.wv["austerite"]

array([ 0.5834239 , -0.3957622 , -0.18323709,  0.49930328, -0.65007716,
        0.04816964,  1.4403331 ,  0.20748572, -0.59815043, -0.11107156,
        0.34141046,  0.7158929 , -0.78166264, -1.0964745 ,  0.06885222,
       -0.47220626, -0.6488406 ,  1.4893552 ,  0.9907471 , -0.32537475,
        0.06925521, -0.5839897 , -0.23195593, -0.16696565, -0.38590914,
        2.162103  ,  1.352613  , -0.37430894,  0.23594747, -0.9440486 ,
       -0.9379771 ,  0.42327824], dtype=float32)

### Calculer la similarité entre deux termes

In [40]:
model.wv.similarity("austerite", "rigueur")

0.66331625

### Chercher les mots les plus proches d'un terme donné

In [36]:
model.wv.most_similar("greve", topn=10)

[('greve_generale', 0.9117452502250671),
 ('ruhr', 0.8361386656761169),
 ('rhenanie', 0.8178145885467529),
 ('population_civile', 0.8096165060997009),
 ('region_parisienne', 0.8090487122535706),
 ('continuation', 0.806405782699585),
 ('catastrophe', 0.7975539565086365),
 ('reddition', 0.790824294090271),
 ('majeure_partie', 0.7888742089271545),
 ('negociation', 0.7881587743759155)]

In [34]:
model.wv.most_similar("politique", topn=10)

[('doctrine', 0.919273853302002),
 ('dictature', 0.9121535420417786),
 ('solidarite', 0.901498019695282),
 ('politique_etrangere', 0.8978065252304077),
 ('neutralite', 0.8976921439170837),
 ('politique_exterieure', 0.8921958208084106),
 ('cooperation', 0.8871824145317078),
 ('coalition', 0.8844858407974243),
 ('democratique', 0.881001353263855),
 ('discipline', 0.8786153793334961)]

In [33]:
model.wv.most_similar("recession", topn=10)

[('conjoncture', 0.9383889436721802),
 ('production_nationale', 0.9181608557701111),
 ('politique_allemande', 0.9134177565574646),
 ('nouvelle_phase', 0.9112986326217651),
 ('gestation', 0.9100854992866516),
 ('crise_financiere', 0.9072067141532898),
 ('tendance_generale', 0.9011976718902588),
 ('situation_interieure', 0.9003405570983887),
 ('norme', 0.8937374949455261),
 ('consultation_electorale', 0.8931797742843628)]

In [41]:
model.wv.most_similar("rigueur", topn=10)

[('prudence', 0.9184521436691284),
 ('contradiction', 0.9182908535003662),
 ('severite', 0.9146202206611633),
 ('realite', 0.9113166332244873),
 ('consideration', 0.9045991897583008),
 ('moderation', 0.8964959979057312),
 ('sagesse', 0.8936125040054321),
 ('logique', 0.8915309309959412),
 ('gravite', 0.8904096484184265),
 ('confusion', 0.8875055909156799)]

In [ ]:
model.wv.most_similar("austerite", topn=10)

[('heredite', 0.9467633366584778),
 ('exageration', 0.9312036633491516),
 ('instabilite', 0.9181275367736816),
 ('incomprehension', 0.91788250207901),
 ('intransigeance', 0.9132502675056458),
 ('etrangete', 0.91274493932724),
 ('inspiration', 0.911300539970398),
 ('ampleur', 0.9103581309318542),
 ('hypocrisie', 0.9068829417228699),
 ('incoherence', 0.9067530035972595)]

### Faire des recherches complexes à travers l'espace vectoriel

In [42]:
print(model.wv.most_similar(positive=['politique', 'economique', 'recession'], negative=['austerite']))

[('situation_economique', 0.9120184183120728), ('vie_economique', 0.9004865884780884), ('frontiere_linguistique', 0.8914955854415894), ('communaute_europeenne', 0.8872699737548828), ('paix_sociale', 0.8789765238761902), ('securite_collective', 0.8770690560340881), ('situation_financiere', 0.8766070008277893), ('situation_generale', 0.8616084456443787), ('situation_actuelle', 0.8590925335884094), ('consolidation', 0.8574387431144714)]
